### Import Model Sklearn & NLP to fit value text

In [1]:
# from pythainlp import word_tokenize
# from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
# from sklearn.decomposition import PCA
# from pythainlp.word_vector import thai2vec 
import numpy as np
import pandas as pd
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from pythainlp.tokenize import word_tokenize,sent_tokenize
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from scipy.cluster import hierarchy
import dill as pickle
import pandas as pd
import pythainlp.word_vector
# model = word_vector.get_model()
model_path = 'thwiki_data/models/'

### read date problem

In [ ]:
df = pd.read_csv("data/dataReV5.csv")
df

In [ ]:
text = df['Problem'].dropna().rename(columns = {"Problem": "problem"})
text

### Import NLP Spit text

In [ ]:
from pythainlp import word_tokenize, Tokenizer

In [ ]:
sentences_newmm= []
sentences_longest= []

for x in text:
    proc_newmm = word_tokenize(x, engine='newmm', keep_whitespace=False)
    proc_longest = word_tokenize(x, engine='longest', keep_whitespace=False)

    lst_newmm = []
    lst_longest = []
    
    for word in proc_newmm:
        if(word != " "):
            lst_newmm.append(word)
    sentences_newmm.append(lst_newmm)
    for word in proc_longest:
        if(word != " "):
            lst_longest.append(word)
    sentences_longest.append(lst_longest)

In [ ]:
model_nlp = pythainlp.word_vector.get_model()

In [ ]:
def sentence_vectorizer(ss,dim=300,use_mean=True): # ประกาศฟังก์ชัน sentence_vectorizer
    s = word_tokenize(ss)
    vec = np.zeros((1,dim))
    for word in s:
#         if word in model_nlp.wv.index2word: 
#             vec+= model_nlp.wv.word_vec(word)
        if word in model_nlp.index2word: 
            vec+= model_nlp.word_vec(word)
        else: pass
    if use_mean: vec /= len(s)
    return vec

lst_vector = []
for i in text:
    lst_vector.append(sentence_vectorizer(i)[0])
X=np.array(lst_vector)
print(X)
# text

In [ ]:
import matplotlib.pyplot as plt
wcss = []
for i in range(1,11):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)

###  Import  scipy.spatial  distance

In [ ]:
from scipy.spatial import distance
import numpy.matlib

In [ ]:
def elbow_method(data,k):
    clusters = [KMeans(n_clusters = c,init = 'k-means++').fit(data) for c in k]
    centr_lst = [cc.cluster_centers_ for cc in clusters]
    k_distance = [distance.cdist(data, cent, 'euclidean') for cent in centr_lst]
    clust_indx = [np.argmin(kd,axis=1) for kd in k_distance]
    distances = [np.min(kd,axis=1) for kd in k_distance]
    avg_within = [np.sum(dist)/data.shape[0] for dist in distances]
    return avg_within

In [ ]:
def elbow_point(curve):
    n_points = len(curve)
    all_coord = np.vstack((range(n_points), curve)).T
    np.array([range(n_points), curve])
    first_point = all_coord[0]
    line_vec = all_coord[-1] - first_point
    line_vec_norm = line_vec / np.sqrt(np.sum(line_vec**2))
    vec_from_first = all_coord - first_point
    scalar_product = np.sum(vec_from_first * np.matlib.repmat(line_vec_norm, n_points, 1), axis=1)
    vec_from_first_parallel = np.outer(scalar_product, line_vec_norm)
    vec_to_line = vec_from_first - vec_from_first_parallel
    dist_to_line = np.sqrt(np.sum(vec_to_line ** 2, axis=1))
    print(dist_to_line)
    idx_best = np.argmax(dist_to_line)
    return idx_best

In [ ]:
elbow=elbow_point(wcss)
elbow.dtype

In [ ]:
n = len(wcss)+1
K= range(1, n)

print("Optimal Cluster Number: ",K[elbow])
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(K, wcss, 'bx-')
ax.plot(K[elbow], wcss[elbow], marker='o', markersize=12,markeredgewidth=2, markeredgecolor='r', markerfacecolor='None')
# ax.set_xticks(np.arange(0,n,1))
# ax.set_yticks(np.arange(0,max(wcss),0.5))
plt.grid()
plt.xlabel('Number of clusters')
plt.ylabel('Average within-cluster sum of squares')
plt.title('Elow for KMeans clustering')
plt.show();

In [ ]:
n_clusters = K[elbow]
clf = KMeans(n_clusters=n_clusters,
            max_iter=100,
            init='k-means++',
            n_init=1)
labels = clf.fit_predict(X)
labels

In [ ]:
n_clusters = K[elbow]
clf = KMeans(n_clusters=n_clusters,
            max_iter=100,
            init='k-means++',
            n_init=1)
labels = clf.fit_predict(X)
labels
cluster = []
value = []
o = ""
for index, sentence in enumerate(sentences_newmm):
#     print(str(labels[index]) + ":" + str(sentence))
    cluster.append(labels[index])
#     value.append(o.join(sentences_newmm[index]))
#     list_of_cluster = list(zip(cluster))
text_cluster = pd.DataFrame({"cluster":cluster})  


In [ ]:
text_cluster

In [ ]:
# df_cluster = pd.DataFrame(list_of_cluster, columns = ['cluster', 'text'])  
df_cluster = df [['Problem','score']]
df_cluster.append(text_cluster, ignore_index = True)
df_cluster["cluster"] = text_cluster.values

In [ ]:
# df_cluster = pd.DataFrame(list_of_cluster, columns = ['cluster', 'text'])  
df_cluster

###  Speech Recognition

In [ ]:
import speech_recognition as sr
# Record Audio
r = sr.Recognizer()
m = sr.Microphone()

In [ ]:
#set threhold level
with m as source: r.adjust_for_ambient_noise(source)
print("Set minimum energy threshold to {}".format(r.energy_threshold))

# Speech recognition using Google Speech Recognition
def checkspeech(r):
#     file_wav
#     sr.Microphone()  sr.WavFile(file_wav)
#     with sr.WavFile("C:/Users/Pack.Apichart/Desktop/re/t1.wav")
    print("Wake UP!!")
    with  sr.Microphone()  as source:
        audio = r.listen(source)
    try:
    # for testing purposes, we're just using the default API key
    # to use another API key, use `r.recognize_google(audio, key="GOOGLE_SPEECH_RECOGNITION_API_KEY")`
    # instead of `r.recognize_google(audio)`

# "en-US"
# "th-TH"
        print("You said: " + r.recognize_google(audio,language = "th-TH"))
        return r.recognize_google(audio,language = "th-TH")
#         return (r.recognize_google(audio))
    except sr.UnknownValueError:
        print("Google Speech Recognition could not understand audio")
        return "stop now"
    except sr.RequestError as e:
        print("Could not request results from Google Speech Recognition service; {0}".format(e))
        return "Could not request results from Google Speech Recognition service"



In [ ]:
# speech = str(checkspeech(r))

In [ ]:
# speech_list = []
# speech_list.append(sentence_vectorizer(str(speech))[0])
# speech_vector = np.array(speech_list)
# result = clf.predict(speech_vector)
# result

### Weigth score Text

In [ ]:
# speech = str(checkspeech(r))

In [ ]:
# speech_list = []
# speech_list.append(sentence_vectorizer(str(speech))[0])
# speech_vector = np.array(speech_list)
# speech_result = clf.predict(speech_vector)
# speech_result

### Weigth score Text

In [ ]:
# def sentence_similarity(s1,s2):
#     for i in s2:
#         print(cosine_similarity(sentence_vectorizer(str(s1)),sentence_vectorizer(str(i))))
# #     return cosine_similarity(sentence_vectorizer(str(s1)),sentence_vectorizer(str(s2)))


In [ ]:
### Function weigth score

In [ ]:
def sentence_similarity(s1,s2):
    for i in s2:
        cosine_similarity(sentence_vectorizer(str(s1)),sentence_vectorizer(str(i)))
    return cosine_similarity(sentence_vectorizer(str(s1)),sentence_vectorizer(str(s2)))


In [ ]:
# print(sentence_similarity("มีเวลาน้อย","มีเวลาพักไม่พอต่อคิว"))
# print(sentence_similarity("มีเวลาน้อย","มีเวลาน้อยไม่สามารถต่อคิวได้"))
# print(sentence_similarity("มีเวลาน้อย","มีเวลาพักกินข้าวน้อย"))
# print(sentence_similarity("มีเวลาน้อย","ร้านอาหารมีจำนวนไม่มากพอที่จะรับจำนวนนักศึกษา"))
# print(sentence_similarity("มีเวลาน้อย","ร้านอาหารมีคิวเยอะ"))


In [ ]:
df_cluster[df_cluster['cluster']==speech_result[0]]

In [ ]:
text2222 = ["รอคิวนาน", "เบื่อเมนูอาหาร", "คิวเยอะ", "อาหารหมด"]
group_text = []
l = []

for y in text2222:
    l.append(sentence_vectorizer(y)[0])
KL=np.array(l)
labels_newProblem = clf.predict(KL)
group_text.append(labels_newProblem)

for countgroup in range(len(labels_newProblem)):
    state_score = ""
    state_similarity = 0
    for i in df_cluster[df_cluster['cluster']==labels_newProblem[countgroup]].values:
        check_similarity = sentence_similarity(text2222[countgroup],i[0])
        if(check_similarity > state_similarity):
            state_similarity = check_similarity
            state_score = i[1]

#         print(check_similarity,i[0],": score",i[1])
    print(text2222[countgroup], "group: ", labels_newProblem[countgroup], "similarity: " ,state_similarity[0][0], "score :" , state_score)
   

Vehicle	restaurant	food delivery	Problem	score	work	เวลาในการรออาหาร	ราคาอาหาร

In [ ]:
df_cluster[df_cluster['cluster']==labels_newProblem[0]]

In [ ]:
print(sentence_similarity("มีเวลาน้อย","มีเวลาพักกินข้าวน้อย"))
l = []

# l.append(sentence_vectorizer("ถ้าร้านนั้นคนเยอะก็จะไปเลือกร้านอื่น")[0])
l.append(sentence_vectorizer("มีเวลาน้อย")[0])

KL=np.array(l)
KL
labels = clf.predict(KL)
print(labels)

### WRITE SCORE TO DATA SET

In [ ]:
#  new_df = pd.DataFrame({
#         'Vehicle' : [''],
#         'restaurant' : [''],
#         'food delivery' : [''],
#         'Problem' : [labels_newProblem[countgroup]],
#         'score' : [state_score],
#         'work' : [''],
#         'เวลาในการรออาหาร' : [''],
#         'ราคาอาหาร' : ['']
#     })
#     new_df.to_csv('data/dataReV5.csv', mode='a', header=False)